<a href="https://colab.research.google.com/github/ykenkou001/Pytorch-ni-yoru-hatten-deeplearning/blob/main/2_4_5_SSD_model_forward_COPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.4 ネットワークモデルの実装、2.5 順伝搬関数の実装

本ファイルでは、SSDのネットワークモデルと順伝搬forward関数を作成します。


# 2.4 学習目標

1.	SSDのネットワークモデルを構築している4つのモジュールを把握する
2.	SSDのネットワークモデルを作成できるようになる
3.	SSDで使用する様々な大きさのデフォルトボックスの実装方法を理解する



# 2.5 学習目標

1.	Non-Maximum Suppressionを理解する
2.	SSDの推論時に使用するDetectクラスの順伝搬を理解する
3.	SSDの順伝搬を実装できるようになる


# 事前準備


とくになし

In [3]:
# パッケージのimport
from math import sqrt
from itertools import product

import pandas as pd
import torch
from torch.autograd import Function
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

# vggモジュールを実装

In [15]:
# 35層にわたる、vggモジュールを作成
def make_vgg():
  layers = [] # 特徴マップを格納する　list
  in_channels = 3 # 色チャネル数

  # vggモジュールで使用する畳み込み層やmaxpoolingのchannel数
  cfg = [64, 64, 'M', 128, 128, 'M', 256, 256,
         256, 'MC', 512, 512, 512, 'M', 512, 512, 512]

  for v in cfg:
    if v == 'M':
      # MaxPoolingLayerの定義
      layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
    elif v == 'MC':
      # ceilは出力サイズを計算結果(float)に対して、切り上げて整数にするモード
      # defaultではoutput sizeを計算結果(float)に対して、切り下げて整数にするfloor　Mode
      layers += [nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)]
    else:
      # 各畳み込み層の定義（ReLUを付け足す）
      conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
      layers += [conv2d, nn.ReLU(inplace=True)]
      in_channels = v
  # 18層目のmaxpooling層の定義
  pool5 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
  # 19層目の定義
  conv6 = nn.Conv2d(512, 1024, kernel_size=3, padding=6, dilation=6)
  # 20層目の定義
  conv7 = nn.Conv2d(1024, 1024, kernel_size=1)
  # 18層目以降の層と畳み込み層に活性化関数を追加しlayersに追加
  layers += [pool5, conv6, 
             nn.ReLU(inplace=True), conv7, nn.ReLU(inplace=True)]
  return nn.ModuleList(layers)

# Operatin Check
vgg_test = make_vgg()
print(vgg_test)

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (17): Conv2d(256, 512, kernel_siz

# extrasモジュールを実装

In [ ]:
# 8層にわたる、extrasモジュールを作成
def make_extras():
  # 各層を格納するlist
  layers = []
  in_channels = 1024 # vggモジュールから出力されたextrasに入力される画像チャネル数

  # extraモジュールの畳み込み層のチャネル数を設定するconfiguration
  cfg = [256, 512, 128, 256, 128, 256, 128, 256]

  layers += [nn.Conv2d(in_channels, cfg[0], kernel_size=(1))]
  layers += [nn.Conv2d(cfg[0], cfg[1], kernel_size=(3), stride=2, padding=1)]
  layers += [nn.Conv2d(cfg[1], cfg[2], kernel_size=(1))]
  layers += [nn.Conv2d(cfg[2], cfg[3], kernel_size=(3), stride=2, padding=1)]
  layers += [nn.Conv2d(cfg[3], cfg[4], kernel_size=(1))]
  layers += [nn.Conv2d(cfg[4], cfg[5], kernel_size=(3))]
  layers += [nn.Conv2d(cfg[5], cfg[6], kernel_size=(1))]
  layers += [nn.Conv2d(cfg[6], cfg[7], kernel_size=(3))]



# locモジュールとconfモジュールを実装

# L2Norm層を実装

# デフォルトボックスを実装

# SSDクラスを実装

# ここから2.5節 順伝搬の実装です

# 関数decodeを実装する

# Non-Maximum Suppressionを行う関数を実装する

# Detectクラスを実装する

# SSDクラスを実装する

以上